In [1]:
# Import packages

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import pandas as pd
from scipy.signal import medfilt
import functools
from scipy.optimize import minimize, bisect
import os
import seaborn as sns

In [2]:
pwd

u'/Users/indre/Desktop/Thesis Data Processing'

In [3]:
cd '/Users/indre/Python/dp_python-master'

/Users/indre/Python/dp_python-master


In [4]:
from dpcore import dp

In [5]:
%matplotlib qt

In [6]:
# Define functions
# From Sylvester et al. (2019)

def compute_derivatives(x,y):
    """function for computing first derivatives of a curve (centerline)
    x,y are cartesian coodinates of the curve
    outputs:
    dx - first derivative of x coordinate
    dy - first derivative of y coordinate
    ds - distances between consecutive points along the curve
    s - cumulative distance along the curve"""
    dx = np.gradient(x) # first derivatives
    dy = np.gradient(y)   
    ds = np.sqrt(dx**2+dy**2)
    s = np.cumsum(ds)
    s = np.hstack((0,np.cumsum(ds[1:])))
    return dx, dy, ds, s


def compute_curvature(x,y):
    dx = np.gradient(x); dy = np.gradient(y)  # first derivatives    
    ds = np.sqrt(dx**2+dy**2)
    ddx = np.gradient(dx); ddy = np.gradient(dy) # second derivatives 
    curvature = (dx*ddy - dy*ddx) / ((dx**2 + dy**2)**1.5)
    s = np.cumsum(ds)
    return curvature, s 

def compute_curvature(x,y):
    """function for computing first derivatives and curvature of a curve (centerline)
    x,y are cartesian coodinates of the curve
    outputs:
    dx - first derivative of x coordinate
    dy - first derivative of y coordinate
    ds - distances between consecutive points along the curve
    s - cumulative distance along the curve
    curvature - curvature of the curve (in 1/units of x and y)"""
    dx = np.gradient(x) # first derivatives
    dy = np.gradient(y)      
    ds = np.sqrt(dx**2+dy**2)
    ddx = np.gradient(dx) # second derivatives 
    ddy = np.gradient(dy) 
    curvature = (dx*ddy - dy*ddx) / ((dx**2 + dy**2)**1.5) # look into this formula more
    s = np.cumsum(ds)
    s = np.hstack((0,np.cumsum(ds[1:])))
    return curvature, s
    
def resample_curve(x,y,deltas,sf):
    dx,dy,ds,s = compute_derivatives(x,y)
    tck, u = scipy.interpolate.splprep([x,y],s=sf) 
    unew = np.linspace(0,1,1+s[-1]/deltas) # vector for resampling
    out = scipy.interpolate.splev(unew,tck) # resampling
    return out[0], out[1]

def correlate_clines(x1,x2,y1,y2):
    # use dynamic time warping to correlate centerlines
    c = len(x1)
    r = len(x2)
    sm = np.zeros((r,c))
    for i in range(0,r):
        sm[i,:] = ((x1-x2[i])**2 + (y1-y2[i])**2)**0.5
    p,q,C,phi = dp(sm,penalty=0.0,gutter=0.0)
    return p,q,sm


In [7]:
#    Define migration functions
# From Sylvester et al. (2019)

def get_migr_dist(x1,y1,x2,y2,years):
    """use dynamic time warping to correlate centerlines
    inputs:
    x1, y1 - coordinates of first centerline
    x2, y2 - coordinates of second centerline
    years - time between the two centerlines, in years
    penalty - parameter that forces more parallel correlation (or not)
    outputs:
    migr_dist - migration rate (in m/years)
    migr_sign - migration sign
    p - indices of correlation in second centerline
    q - indices of correlation in first centerline"""
    p,q,sm = correlate_clines(x1,x2,y1,y2)
    qn = np.delete(np.array(q),np.where(np.diff(q)==0)[0]+1)
    pn = np.delete(np.array(p),np.where(np.diff(q)==0)[0]+1)
    xa = x1[:-1]
    xb = x1[1:]
    ya = y1[:-1]
    yb = y1[1:]
    x = x2[pn][1:]
    y = y2[pn][1:]
    migr_sign = np.sign((x-xa)*(yb-ya) - (y-ya)*(xb-xa))
    migr_dist = migr_sign*sm[pn,qn][1:]/years
    migr_dist = np.hstack((migr_dist[0],migr_dist))
    return migr_dist, migr_sign, p, q
  

def get_predicted_migr_dist(curvature,W,k,Cf,D,kl,s):
    """function for calculating predicted migration rate
    using the simplified Howard-Knutson model
    inputs:
    W - channel width (m)
    k - constant (=1)
    Cf - friction factor
    D - channel depth (m)
    kl - migration constant (m/year)
    s - along-channel distance (m)
    output:
    R1 - predicted migration rate"""
    ds = np.diff(s)
    alpha = k*2*Cf/D
    ns = len(s)
    R0 = kl*W*curvature # preallocate vector for nominal channel migration rate
    R1 = np.zeros(ns) # preallocate adjusted channel migration rate
    for i in range(0,len(R1)):
        si2 = np.hstack((0,np.cumsum(ds[i-1::-1])))  # distance along centerline, backwards from current point 
        G = np.exp(-alpha*si2) # weighting function   
        R1[i] = -1*R0[i] + 2.5*np.sum(R0[i::-1]*G)/np.sum(G) # actual migration rate (m/year)
    return R1

# function for optimizing for Cf:
def get_friction_factor(Cf,curvature,migr_dist,kl,W,k,D,s):
    R1 = get_predicted_migr_dist(curvature,W,k,Cf,D,kl,s)
    corr = correlate(R1, migr_dist)
    # delta time array to match xcorr:
    delta_t = np.arange(1-len(R1), len(R1))
    time_shift = delta_t[corr.argmax()]
    return time_shift # goal is to minimize the time shift

from __future__ import division
import numpy as np


def zerocross1d(x, y, getIndices=False):
    """
    Find the zero crossing points in 1d data.

    Find the zero crossing events in a discrete data set.
    Linear interpolation is used to determine the actual
    locations of the zero crossing between two data points
    showing a change in sign. Data point which are zero
    are counted in as zero crossings if a sign change occurs
    across them. Note that the first and last data point will
    not be considered whether or not they are zero. 

    Parameters
    ----------
    x, y : arrays
        Ordinate and abscissa data values.
    getIndices : boolean, optional
        If True, also the indicies of the points preceding
        the zero crossing event will be returned. Defeualt is
        False.

    Returns
    -------
    xvals : array
        The locations of the zero crossing events determined
        by linear interpolation on the data.
    indices : array, optional
        The indices of the points preceding the zero crossing
        events. Only returned if `getIndices` is set True.
    """

    # Indices of points *before* zero-crossing
    indi = np.where(y[1:]*y[0:-1] < 0.0)[0]

    # Find the zero crossing by linear interpolation
    dx = x[indi+1] - x[indi]
    dy = y[indi+1] - y[indi]
    zc = -y[indi] * (dx/dy) + x[indi]

    # What about the points, which are actually zero
    zi = np.where(y == 0.0)[0]
    # Do nothing about the first and last point should they
    # be zero
    zi = zi[np.where((zi > 0) & (zi < x.size-1))]
    # Select those point, where zero is crossed (sign change
    # across the point)
    zi = zi[np.where(y[zi-1]*y[zi+1] < 0.0)]

    # Concatenate indices
    zzindi = np.concatenate((indi, zi)) 
    # Concatenate zc and locations corresponding to zi
    zz = np.concatenate((zc, x[zi]))

    # Sort by x-value
    sind = np.argsort(zz)
    zz, zzindi = zz[sind], zzindi[sind]

    if not getIndices:
        return zz
    else:
        return zz, zzindi

In [8]:
# Import files and vector data


# Import CVS Files for seafloor
df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/seafloor_centerline_UTM_15N.csv', usecols=[0,1])
df.columns = ['x', 'y']

x0r = np.array(df['x'])
y0r = np.array(df['y'])

# Import CVS Files for HZ 1 and HZ2

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/thalweg_hz1_vertices_v3.csv', usecols=[0,1])
df.columns = ['x', 'y']
# df[:5]

x1r = np.array(df['x'])
y1r = np.array(df['y'])

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/thalweg_hz2_vertices_v4.csv',usecols=[0,1])
df.columns = ['x', 'y']

x2r = np.array(df['x'])
y2r = np.array(df['y'])

# Import CVS Files for HZ 3 and HZ4

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/thalweg_hz3_vertices2.csv',usecols=[0,1])
df.columns = ['x', 'y']

x3r = np.array(df['x'])
y3r = np.array(df['y'])
# df[:5]

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/thalweg_hz4_vertices.csv',header=None,usecols=[0,1])
df.columns = ['x', 'y']

x4r = np.array(df['x'])
y4r = np.array(df['y'])

# Map of unsmoothed centerlines

plt.figure(figsize=(8,6))

plt.plot(x4r,y4r, 'k')
plt.plot(x3r,y3r, 'k')
plt.plot(x2r,y2r, 'k')
plt.plot(x1r,y1r, 'k')
plt.plot(x0r,y0r, 'k')

plt.axis('equal')
plt.ylabel('distance from south to north (m)', fontsize =12); 
plt.xlabel('distance from east to west (m)',fontsize =12)
plt.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))

In [9]:
# Smooth, resample vector data

import scipy.interpolate

deltas = 25.0 # sampling distance, previously and in half width analysis is 50m
sf = 20000 # smoothing factor changed from 200000 and then half that and then 20000
    
x0, y0 = resample_curve(x0r,y0r,deltas,sf)
x1, y1 = resample_curve(x1r,y1r,deltas,sf)
x2, y2 = resample_curve(x2r,y2r,deltas,sf)
x3, y3 = resample_curve(x3r,y3r,deltas,sf)
x4, y4 = resample_curve(x4r,y4r,deltas,sf)

# Map of resampled smoothed centerlines

plt.plot(x4r,y4r,'k')
plt.plot(x3r,y3r,'k')
plt.plot(x2r,y2r,'k')
plt.plot(x1r,y1r,'k')
plt.plot(x0r,y0r,'k')

plt.plot(x4,y4,'y')
plt.plot(x3,y3,'c')
plt.plot(x2,y2,'g')
plt.plot(x1,y1,'b')
plt.plot(x0,y0,'k')

plt.ylabel('distance from south to north (m)', fontsize =12); 
plt.xlabel('distance from east to west (m)',fontsize =12)
plt.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))

/Users/indre/anaconda3/envs/DTW/lib/python2.7/site-packages/ipykernel_launcher.py:49: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.




# Analyze Timestep 1 - Horizon 4 to and Horizon 3

In [11]:
# 1. Compute, Smooth Curvature

curv,s = compute_curvature(x4,y4) # Horizon 4
from scipy.signal import savgol_filter # Plot Smoothed and Raw Curvature
curv_s = savgol_filter(curv,51,3) # Mess with "51" or middle figure to play w smoothing (changed to 31)


In [12]:
# 2. Compute, Smooth Migration Distance

years = 1.0 #to get migration distance because we dont know the rate
migr_dist, migr_sign, p, q = get_migr_dist(x4,y4,x3,y3,years)

migr_dist = medfilt(savgol_filter(migr_dist,51,3),kernel_size=5) # smoothing

curv = medfilt(savgol_filter(curv,51,3),kernel_size=5) # smoothing

loc_zero_curv_s, loc_zero_curv = zerocross1d(s, curv, getIndices=True)
loc_zero_migr_s, loc_zero_migr = zerocross1d(s, migr_dist, getIndices=True)


In [12]:
#  Run a segment like this  to crop out incomplete contour data, based on the guidelines below

# Hz3 begins at index 1601 and ends at 5055 for ts2
# Hz4 beg at index 0 and ends at index 3250 for ts 1
# for ts 2, hz 2 beg at index 1376 and ends in 6571
# for ts C, hz 1 beg at 1437 and end at 6788
# for ts C, hz 3 end at 6788
# for ts A and B, hz 4 ends with 3389
# fo time step A, hz1 [3137:6942]
# time step B, hz 2 [3018:6722]

# migr_dist = migr_dist[:3250]
# curv = curv[:3250]
# s = s[:3250] 

In [154]:
# 3. Plot Migration and Curvature

fig = plt.figure(figsize=(30,5))
ax1 = fig.add_subplot(111)
ax1.plot(s/1000.0,curv) 

ax1.set_xlim(0,max(s/1000.0))
ax1.set_ylabel('curvature (1/m)', fontsize=14)

ax1.plot([0,max(s/1000)],[0,0],'k--')

ax2 = ax1.twinx()
ax2.plot(s/1000.0,migr_dist,'r')
ax1.set_xlabel('distance along channel (km)', fontsize=14)

ax2.set_ylabel('migration rate (m/year)', fontsize=14, color='r')
ax2.set_ylim(-300,300)
ax1.set_ylim(-0.003,0.003);

plt.title('Migration from Horizon 4 to Horizon 3', fontsize =16);

In [155]:
# 4. Import bends* from CSV 

# To edit bends, see "bend_picker" in /Desktop/Thesis Data Processing and change file description below to "..._NEW"
# For smoothing factor on curv at '31' use v2, for '51' use  v3

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/Joshua_hz4_to_hz3_inflection_and_zero_migration_indices_v2.csv', usecols=[0,1,2,3,4])
df.columns = ['bend','index_inflection_point','index_zero_migration','s_coordinate_index_ip','s_coordinate_index_zm']
                 
BEND = np.array(df['bend'])
LZC = np.array(df['index_inflection_point'])
LZM = np.array(df['index_zero_migration'])
LZC_s = np.array(df['s_coordinate_index_ip'])
LZM_s = np.array(df['s_coordinate_index_zm'])

BEND=BEND+26  # numbering bends

LZC=LZC.astype(int)
LZM=LZM.astype(int)
 

In [174]:
# 5. Find lag.

# Print rough lag estimate from inflection points.

print np.median(LZM-LZC) # in index points
print deltas*np.median(LZM-LZC) #in meters

22.0
550.0


In [13]:
# To calculate final lag in the paper,
# iterate index point values in the segments below to maximize correlation, 
# starting with the first value (index #) in the rough estimate above.

#Without lag
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, _ = pearsonr(curv, migr_dist)
print('Pearsons correlation: %.3f' % corr)

#With lag
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, _ = pearsonr(curv[:-28], migr_dist[28:])
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.460
Pearsons correlation: 0.634


In [180]:
# Convert the final lag to meters using the number found above in place of "28"

lag = 28*deltas
print lag

700.0


In [63]:
# 6. Correlation plot, 
# use lag in place of 28

# This part is necessary for color coding low quality data in yellow.
curv1 = curv[:-28]
migr1= migr_dist[28:]

plt.figure(figsize=(8,8)) 
sns.kdeplot(curv[:-28], migr_dist[28:],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)
plt.scatter(curv1[417:3759][::20],migr_dist[417:3759][::20],c='k',s=10)
plt.scatter(curv[:417-29][::20],migr_dist[29:417][::20],c='y',s=10)
plt.scatter(curv[3759:-29][::20],migr_dist[3759+29:][::20],c='y',s=10)
plt.ylabel('migration distance (m), with phase lag', fontsize=14)

Text(0,0.5,'migration distance (m), with phase lag')

In [29]:
# 7. Map bends, cropped 

# colors: hz1 - 'b', hz 2 - 'g' , hz3 - 'c', hz 4 - 'y'

plt.figure(figsize=(60,60))
           
plt.plot(x4[:4511],y4[:4511], 'y')
plt.plot(x3[1606:],y3[1606:],'c')

plt.axis('equal')
plt.scatter(x4[LZC], y4[LZC], c='r')
plt.scatter(x4[LZM], y4[LZM], c='b')  

for i, txt in enumerate(BEND):
    plt.annotate(txt, (x4[LZC][i], y4[LZC][i]), color='r',  size='small')
for i, txt in enumerate(BEND):
    plt.annotate(txt, (x4[LZM][i], y4[LZM][i]), color='b', size='small')
    
LZM=LZM.astype(int)
LZC=LZC.astype(int)
plt.ylabel('distance from south to north (m)', fontsize =12); 
plt.xlabel('distance from east to west (m)',fontsize =12)
plt.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))

In [ ]:
# 8. Let's plot data by bends

W = 174

fig, ax1 = plt.subplots(figsize=(16,4))

y1g = .45 # Top for curv
y2g = 0.0 # Zero for curv
y3g = -0.58332 # Zero for migr
y4g = -1.1 # Bottom for curv

for i in range(1,len(LZC)-1,2):
    xcoords = [LZC_s[i],LZC_s[i+1],LZC_s[i+1],LZM_s[i+1],LZM_s[i+1],LZM_s[i],LZM_s[i],LZC_s[i]]
    ycoords = [y1g,y1g,y2g,y3g,y4g,y4g,y3g,y2g]
    ax1.fill(xcoords,ycoords,color=[0.85,0.85,0.85],zorder=0) 
        
offset = 10
deltas = 25.0

ax1.fill_between(s, 0, curv*W) 
ax2 = ax1.twinx()
ax2.fill_between(s, 0, migr_dist, facecolor='green')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y4g,y1g)
ax2.set_ylim(-400,800)
ax1.set_xlim(0,s[-1])
   
ax1.set_ylabel('W/r (m)')
ax2.set_ylabel('migration distance (m)')
ax1.set_xlabel("distance along channel axis (m)")

plt.tight_layout()


# Analyze Timestep 2 - Horizon 3 to Horizon 2

In [14]:
# 1. Compute, Smooth Curvatures

curv,s = compute_curvature(x2,y2) # Horizon 2

from scipy.signal import savgol_filter#Plot Smoothed and Raw Curvature
curv_s = savgol_filter(curv,51,3) # mess with "51" or middle figure to play w smoothing 

curv,s = compute_curvature(x3,y3) # Horizon 3

from scipy.signal import savgol_filter#Plot Smoothed and Raw Curvature
curv_s = savgol_filter(curv,51,3) # mess with "51" or middle figure to play w smoothing

In [15]:
# 2. Compute, Smooth Migration Distance

years = 1.0 # to get migration distance because we dont know the rate
migr_dist, migr_sign, p, q = get_migr_dist(x3,y3,x2,y2,years)

migr_dist = medfilt(savgol_filter(migr_dist,51,3),kernel_size=5) # smoothing
curv,s = compute_curvature(x3,y3)

curv = medfilt(savgol_filter(curv,51,3),kernel_size=5) # smoothing

loc_zero_curv_s, loc_zero_curv = zerocross1d(s, curv, getIndices=True)
loc_zero_migr_s, loc_zero_migr = zerocross1d(s, migr_dist, getIndices=True)

In [20]:
# # When cropping noncontour data, run this.... 

# migr_dist = migr_dist[:5055]
# curv = curv[:5055]
# s = s[:5055] 


In [28]:
# 3. Plot Migration and Curvature

#hz 3 to hz 2 plot

fig = plt.figure(figsize=(30,5))
ax1 = fig.add_subplot(111)
ax1.plot(s/1000.0,curv) 

ax1.set_xlim(0,max(s/1000.0))
ax1.set_ylabel('curvature (1/m)', fontsize=14)

ax1.plot([0,max(s/1000)],[0,0],'k--') # Label curv_s=0

ax2 = ax1.twinx()
ax2.plot(s/1000.0,migr_dist,'r')
ax1.set_xlabel('distance along channel (km)', fontsize=14)

ax2.set_ylabel('migration rate (m/year)', fontsize=14, color='r')
ax2.set_ylim(-300,300)
ax1.set_ylim(-0.003,0.003);

plt.title('Migration from Horizon 3 to Horizon 2', fontsize =16);

In [191]:
# 4. Import bends* from CSV 
# * To edit, see "bend_picker" in /Desktop/Thesis Data Processing and change file description below to "..._NEW"

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/Joshua_hz3_to_hz2_inflection_and_zero_migration_indices_v2.csv', usecols=[0,1,2,3,4])
df.columns = ['bend','index_inflection_point','index_zero_migration','s_coordinate_index_ip','s_coordinate_index_zm']
                 
BEND = np.array(df['bend'])
LZC = np.array(df['index_inflection_point'])
LZM = np.array(df['index_zero_migration'])
LZC_s = np.array(df['s_coordinate_index_ip'])
LZM_s = np.array(df['s_coordinate_index_zm'])

BEND= BEND+13 # numbering

LZC=LZC.astype(int)
LZM=LZM.astype(int)

In [192]:
# 5. Find lag.

# Lag estimate

print np.median(LZM-LZC)*deltas
print np.median(LZM-LZC)

# use this as a starting point 
# then play with the lag in the pearsons corr
# to find max correlation closest to starting point

657.9545454545454
26.318181818181817


In [16]:
# Iterate for final lag

#Without lag
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, _ = pearsonr(curv, migr_dist)
print('Pearsons correlation: %.3f' % corr)

#With lag!
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, _ = pearsonr(curv[:-25], migr_dist[25:])
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.375
Pearsons correlation: 0.595


In [205]:
# Run to convert lax from index points to meters
25*deltas

625.0

In [15]:
# 6. Correlation plot

plt.figure(figsize=(8,8))
sns.kdeplot(curv[:-26], migr_dist[26:],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)
plt.scatter(curv[:5055-26][::20],migr_dist[26:5055][::20],c='k',s=10)
plt.scatter(curv[5055:-26][::20],migr_dist[5055+26:][::20],c='y',s=10)
plt.xlabel('curvature (1/m)', fontsize=14)
plt.ylabel('migration distance (m), with phase lag', fontsize=14)

Text(0,0.5,'migration distance (m), with phase lag')

In [52]:
# 7. Map bends (cropped) 
# colors: hz1 - 'b', hz 2 - 'g' , hz3 - 'c', hz 4 - 'y'

plt.figure(figsize=(60,60))
           
plt.plot(x3,y3, 'c')
plt.plot(x2[1376:],y2[1376:], 'g')
plt.axis('equal')
plt.scatter(x3[LZC], y3[LZC], c='r')
plt.scatter(x3[LZM], y3[LZM], c='b')  

for i, txt in enumerate(BEND):
    plt.annotate(txt, (x3[LZC][i], y3[LZC][i]), color='r', size='small')
for i, txt in enumerate(BEND):
    plt.annotate(txt, (x3[LZM][i], y3[LZM][i]), color='b', size='small')

LZM=LZM.astype(int)
LZC=LZC.astype(int)
plt.ylabel('distance from south to north (m)', fontsize =12); 
plt.xlabel('distance from east to west (m)',fontsize =12)
plt.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))

In [58]:
# 8. Plot data by bends

W = 174

fig, ax1 = plt.subplots(figsize=(16,4))

y1g = .45 # Top for curv
y2g = 0.0 # Zero for curv
y3g = -0.58332 # Zero for migr
y4g = -1.1 # Bottom for curv

for i in range(0,len(LZC)-1,2):
    xcoords = [LZC_s[i],LZC_s[i+1],LZC_s[i+1],LZM_s[i+1],LZM_s[i+1],LZM_s[i],LZM_s[i],LZC_s[i]]
    ycoords = [y1g,y1g,y2g,y3g,y4g,y4g,y3g,y2g]
    ax1.fill(xcoords,ycoords,color=[0.85,0.85,0.85],zorder=0) 
        
offset = 10
deltas = 25.0

ax1.fill_between(s, 0, curv*W) 
ax2 = ax1.twinx()
ax2.fill_between(s, 0, migr_dist, facecolor='green')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y4g,y1g)
ax2.set_ylim(-300,600)
ax1.set_xlim(0,s[-1])
    
ax1.set_ylabel('W/r (m)')
ax2.set_ylabel('migration distance (m)')
ax1.set_xlabel("distance along channel axis (m)")

plt.tight_layout()
    

# Analyze Timestep 3 - Horizon 2 to and Horizon 1

In [332]:
# Steps
    # 1. Compute, Smooth Curvature for Hz 2, 1 
    # 2. Compute, Smooth Migration for Hz 2, 1 
    # 3. Plot migration and curvature all together.
    # 4. Find lag and correlation (start with Zoltan's process) 
    # 5. Pick bends and save to csv
    # 6. Map bends
    # 7. Plot data by bend

In [31]:
# 1. Compute, Smooth Curvatures
#  Statistics

curv,s = compute_curvature(x1,y1) # Horizon 1

from scipy.signal import savgol_filter#Plot Smoothed and Raw Curvature
curv_s = savgol_filter(curv,51,3) # mess with "51" or middle figure to play w smoothing (changed to 31)

curv,s2 = compute_curvature(x2,y2) # Horizon 2

from scipy.signal import savgol_filter#Plot Smoothed and Raw Curvature
curv_s = savgol_filter(curv,51,3) # mess with "51" or middle figure to play w smoothing (changed to 31)

In [32]:
# 2. Compute, Smooth Migration Distance

years = 1.0 #to get migration distance because we dont know the rate
migr_dist, migr_sign, p, q = get_migr_dist(x2,y2,x1,y1,years)

migr_dist = medfilt(savgol_filter(migr_dist,51,3),kernel_size=5) # smoothing
curv,s = compute_curvature(x2,y2)
curv = medfilt(savgol_filter(curv,41,3),kernel_size=5) # smoothing

loc_zero_curv_s, loc_zero_curv = zerocross1d(s, curv, getIndices=True)
loc_zero_migr_s, loc_zero_migr = zerocross1d(s, migr_dist, getIndices=True)


In [218]:
# 3. Plot Migration and Curvature

#hz 2 to hz 1 plot

fig = plt.figure(figsize=(30,5))
ax1 = fig.add_subplot(111)
ax1.plot(s/1000.0,curv) 

ax1.set_xlim(0,max(s/1000.0))
ax1.set_ylabel('curvature (1/m)', fontsize=14)

#Label curv_s=0
ax1.plot([0,max(s/1000)],[0,0],'k--')

ax2 = ax1.twinx()
ax2.plot(s/1000.0,migr_dist,'r')
ax1.set_xlabel('distance along channel (km)', fontsize=14)

ax2.set_ylabel('migration rate (m/year)', fontsize=14, color='r')
ax2.set_ylim(-300,300)
ax1.set_ylim(-0.003,0.003);

plt.title('Migration from Horizon 2 to Horizon 1', fontsize =16);

In [34]:
# 4. Import bends* from CSV 

# * To edit, see "bend_picker" in /Desktop/Thesis Data Processing and change file description below to "..._NEW"
#SO NOW YOU NEED TO REDO BEND USING ZOLTANS IMPROVED CODE...
# For smoothing factor on curv at '31' use v2, for '51' use  v3

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/Joshua_hz2_to_hz1_inflection_and_zero_migration_indices_v6.csv', usecols=[0,1,2,3,4])
df.columns = ['bend','index_inflection_point','index_zero_migration','s_coordinate_index_ip','s_coordinate_index_zm']
                 
BEND = np.array(df['bend'])
LZC = np.array(df['index_inflection_point'])
LZM = np.array(df['index_zero_migration'])
LZC_s = np.array(df['s_coordinate_index_ip'])
LZM_s = np.array(df['s_coordinate_index_zm'])

# Delete bend zero
BEND = BEND[1:]
LZC = LZC[1:]
LZM = LZM[1:]
LZC_s = LZC_s[1:]
LZM_s = LZM_s[1:]

LZC=LZC.astype(int)
LZM=LZM.astype(int)


In [35]:
# 5. Find lag

# Rough lag estimate

print np.median((LZM-LZC)*deltas)
print np.median(LZM-LZC)

675.0
27.0


In [36]:
# Find final lag

#Without lag
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, _ = pearsonr(curv, migr_dist)
print('Pearsons correlation: %.3f' % corr)

#With lag!
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, _ = pearsonr(curv[:-27], migr_dist[27:])
print('Pearsons correlation: %.3f' % corr)


Pearsons correlation: 0.406
Pearsons correlation: 0.758


In [12]:
# 6. Correlation plots

plt.figure(figsize=(8,8))
sns.kdeplot(curv[:-27], migr_dist[27:],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)
plt.scatter(curv[:-27][::20],migr_dist[27:][::20],c='k',s=10)

plt.xlabel('curvature (1/m)', fontsize=14)
plt.ylabel('migration distance (m), with phase lag', fontsize=14)

Text(0,0.5,'migration distance (m), with phase lag')

In [37]:
# 7. Map bends and plot data by bends

plt.figure(figsize=(60,60))
           
plt.plot(x2,y2,'g')
plt.plot(x1,y1,'b')
plt.axis('equal')
plt.scatter(x2[LZC], y2[LZC], c='r')
plt.scatter(x2[LZM], y2[LZM], c='k')  

for i, txt in enumerate(BEND):
    plt.annotate(txt, (x2[LZC][i], y2[LZC][i]), color='r', size='small')
for i, txt in enumerate(BEND):
    plt.annotate(txt, (x2[LZM][i], y2[LZM][i]), color='k', size='small')
    
LZM=LZM.astype(int)
LZC=LZC.astype(int)
plt.ylabel('distance from south to north (m)', fontsize =12); 
plt.xlabel('distance from east to west (m)',fontsize =12)
plt.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))


In [19]:
# 8. Plot data by bends

W =174

fig, ax1 = plt.subplots(figsize=(16,4))

y1g = .495 # Top for curv
y2g = 0.0 # Zero for curv
y3g = -0.70166 # Zero for migr
y4g = -1.3 # Bottom for curv

for i in range(0,len(LZC)-1,2):
    xcoords = [LZC_s[i],LZC_s[i+1],LZC_s[i+1],LZM_s[i+1],LZM_s[i+1],LZM_s[i],LZM_s[i],LZC_s[i]]
    ycoords = [y1g,y1g,y2g,y3g,y4g,y4g,y3g,y2g]
    ax1.fill(xcoords,ycoords,color=[0.85,0.85,0.85],zorder=0) 
        
offset = 10
deltas = 25.0

ax1.fill_between(s, 0, curv*W) 
ax2 = ax1.twinx()
ax2.fill_between(s, 0, migr_dist, facecolor='green')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y4g,y1g)
ax2.set_ylim(-300,600)
ax1.set_xlim(0,s[-1])
   
ax1.set_ylabel('W/r (m)')
ax2.set_ylabel('migration distance (m)')
ax1.set_xlabel("distance along channel axis (m)")

plt.tight_layout()

NameError: name 'LZC' is not defined

# Analyze seafloor - timestep 4

In [74]:
# 1. Compute, Smooth Curvature

from scipy.signal import savgol_filter #Plot Smoothed and Raw Curvature
curv,s = compute_curvature(x1,y1) # Horizon 1
curv = savgol_filter(curv,51,3) # mess with "51" or middle figure to play w smoothing (changed to 31)


In [75]:
# 2. Compute, Smooth Migration Distance

years = 1.0 #to get migration distance because we dont know the rate
migr_dist, migr_sign, p, q = get_migr_dist(x1,y1,x0,y0,years)
migr_dist = medfilt(savgol_filter(migr_dist,51,3),kernel_size=5) # smoothing
loc_zero_curv_s, loc_zero_curv = zerocross1d(s, curv, getIndices=True)
loc_zero_migr_s, loc_zero_migr = zerocross1d(s, migr_dist, getIndices=True)

In [15]:
# 3. Plot Migration and Curvature

fig = plt.figure(figsize=(30,5))
ax1 = fig.add_subplot(111)
ax1.plot(s/1000.0,curv) 

ax1.set_xlim(0,max(s/1000.0))
ax1.set_ylabel('curvature (1/m)', fontsize=14)

#Label curv_s=0
ax1.plot([0,max(s/1000)],[0,0],'k--')

ax2 = ax1.twinx()
ax2.plot(s/1000.0,migr_dist,'r')
ax1.set_xlabel('distance along channel (km)', fontsize=14)

ax2.set_ylabel('migration rate (m/year)', fontsize=14, color='r')
ax2.set_ylim(-400,400)
ax1.set_ylim(-0.0045,0.0045);

plt.title('Migration from Horizon 1 to Horizon 0', fontsize =16);

In [76]:
# 4. Import bends* from CSV 

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/Joshua_hz1_to_hz0_inflection_and_zero_migration_indices.csv', usecols=[0,1,2,3,4])
df.columns = ['bend','index_inflection_point','index_zero_migration','s_coordinate_index_ip','s_coordinate_index_zm']
                 
BEND = np.array(df['bend'])
LZC = np.array(df['index_inflection_point'])
LZM = np.array(df['index_zero_migration'])
LZC_s = np.array(df['s_coordinate_index_ip'])
LZM_s = np.array(df['s_coordinate_index_zm'])

LZC=LZC.astype(int)
LZM=LZM.astype(int)

In [54]:
# 5. Find lag.

# Rough estimate of lag
print np.median(LZM-LZC)
print np.median(LZM-LZC)*deltas

Pearsons correlation: -0.436
Pearsons correlation: -0.479
300.0


In [ ]:
# Iterate for final lag.
 
# Without lag
from scipy.stats import pearsonr
corr, _ = pearsonr(curv, migr_dist) # Calculate Pearson's correlation


print('Pearsons correlation: %.3f' % corr)

# With lag!
from scipy.stats import pearsonr
corr, _ = pearsonr(curv[:-12], migr_dist[12:]) # Calculate Pearson's correlation
print('Pearsons correlation: %.3f' % corr)

print deltas*12 # Convert from index points to meters


In [45]:
# 6. Correlation plot

plt.figure(figsize=(8,8))
sns.kdeplot(curv[:-12], migr_dist[12:],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)

# If cropping out low quality data, run the below
# plt.scatter(curv[3389:-29][::20],migr_dist[29+3389:][::20],c='y',s=10) 
# plt.scatter(curv[:3389-29][::20],migr_dist[29:3389][::20],c='k',s=10)

plt.scatter(curv[:-12][::20],migr_dist[12:][::20],c='k',s=10)

plt.xlabel('curvature (1/m)', fontsize=14)
plt.ylabel('migration distance (m), with phase lag', fontsize=14)


Text(0,0.5,'migration distance (m), with phase lag')

In [46]:
# 7. Map bends, cropped

plt.figure(figsize=(60,60))
           
plt.plot(x1,y1,'b')
plt.plot(x0,y0, 'k')
plt.axis('equal')
plt.scatter(x1[LZC], y1[LZC], c='b')
plt.scatter(x1[LZM], y1[LZM], c='r')  

for i, txt in enumerate(BEND):
    plt.annotate(txt, (x1[LZC][i], y1[LZC][i]), color='b', size='small')
for i, txt in enumerate(BEND):
    plt.annotate(txt, (x1[LZM][i], y1[LZM][i]), color='r', size='small')
plt.ylabel('distance from south to north (m)', fontsize =12); 
plt.xlabel('distance from east to west (m)',fontsize =12)
plt.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))

In [47]:
# 8. Let's plot data by bends

W = 174

fig, ax1 = plt.subplots(figsize=(16,4))

y1g = .60 # Top for curv
y2g = 0.0 # Zero for curv
y3g = -0.605 # Zero for migr
y4g = -1.2 # Bottom for curv

for i in range(1,len(LZC)-1,2):
    xcoords = [LZC_s[i],LZC_s[i+1],LZC_s[i+1],LZM_s[i+1],LZM_s[i+1],LZM_s[i],LZM_s[i],LZC_s[i]]
    ycoords = [y1g,y1g,y2g,y3g,y4g,y4g,y3g,y2g]
    ax1.fill(xcoords,ycoords,color=[0.85,0.85,0.85],zorder=0) 
        
offset = 10
deltas = 25.0


ax1.fill_between(s, 0, curv*W) 
ax2 = ax1.twinx()
ax2.fill_between(s, 0, migr_dist, facecolor='maroon')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y4g,y1g)
ax2.set_ylim(-300,700)
ax1.set_xlim(0,s[-1])
   
ax1.set_ylabel('W/r (m)')
ax2.set_ylabel('migration distance (m)')
ax1.set_xlabel("distance along channel axis (m)")

plt.tight_layout()
    

# Analyze biggest timestep - time step A

In [38]:
# 1. Compute, Smooth Curvatures

curv,s = compute_curvature(x1,y1) # Horizon 1
from scipy.signal import savgol_filter#Plot Smoothed and Raw Curvature
curv = savgol_filter(curv,51,3) # mess with "51" or middle figure to play w smoothing (changed to 31)


In [39]:
# 2. Compute, Smooth Migration Distance

years = 1.0 #to get migration distance because we dont know the rate
migr_dist, migr_sign, p, q = get_migr_dist(x4,y4,x1,y1,years)
migr_dist = medfilt(savgol_filter(migr_dist,51,3),kernel_size=5) # smoothing
loc_zero_curv_s, loc_zero_curv = zerocross1d(s, curv, getIndices=True)
loc_zero_migr_s, loc_zero_migr = zerocross1d(s, migr_dist, getIndices=True)

In [88]:
#  Run to crop out low quality  contour data

migr_dist = migr_dist[:3389]
curv = curv[:3389]
s = s[:3389] 

In [174]:
# 3. Plot Migration and Curvature

fig = plt.figure(figsize=(30,5))
ax1 = fig.add_subplot(111)
ax1.plot(s/1000.0,curv) 

ax1.set_xlim(0,max(s/1000.0))
ax1.set_ylabel('curvature (1/m)', fontsize=14)

#Label curv_s=0
ax1.plot([0,max(s/1000)],[0,0],'k--')

ax2 = ax1.twinx()
ax2.plot(s/1000.0,migr_dist,'r')
ax1.set_xlabel('distance along channel (km)', fontsize=14)

ax2.set_ylabel('migration rate (m/year)', fontsize=14, color='r')
ax2.set_ylim(-600,600)
ax1.set_ylim(-0.003,0.003);

plt.title('Migration from Horizon 4 to Horizon 1', fontsize =16);

In [97]:
# 4. Import bends* from CSV 

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/Joshua_hz4_to_hz1_inflection_and_zero_migration_indices.csv', usecols=[0,1,2,3,4])
df.columns = ['bend','index_inflection_point','index_zero_migration','s_coordinate_index_ip','s_coordinate_index_zm']
                 
BEND = np.array(df['bend'])
LZC = np.array(df['index_inflection_point'])
LZM = np.array(df['index_zero_migration'])
LZC_s = np.array(df['s_coordinate_index_ip'])
LZM_s = np.array(df['s_coordinate_index_zm'])

# Editing bend assignments, first by matching bend no 1 to Hz 2 to Hz1

BEND=BEND+26  # add [amount] to bend 0 to account for upstream curvatures, match to 2 to 1

LZC=LZC.astype(int)
LZM=LZM.astype(int)

In [83]:
# 5. Lag

# Rough estimate of lag

print np.mean((LZM-LZC)*deltas)
print np.mean(LZM-LZC)

# use mean where data is sparse in timesteps 1,2, A, B but median for rest

762.1212121212121
30.484848484848484


In [40]:
# Find final lag

#Without lag
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, _ = pearsonr(curv, migr_dist)
print('Pearsons correlation: %.3f' % corr)

#With lag!
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, _ = pearsonr(curv[:-29], migr_dist[29:])
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.547
Pearsons correlation: 0.782


In [30]:
# 6. Correlation plot

plt.figure(figsize=(8,8))
sns.kdeplot(curv[:-29], migr_dist[29:],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)

# # Run to color code cropped data
# plt.scatter(curv[3389:-29][::20],migr_dist[29+3389:][::20],c='y',s=10) 
# plt.scatter(curv[:3389-29][::20],migr_dist[29:3389][::20],c='k',s=10)

plt.scatter(curv[:-29][::20],migr_dist[29:][::20],c='k',s=10)

plt.xlabel('curvature (1/m)', fontsize=14)
plt.ylabel('migration distance (m), with phase lag', fontsize=14)

Text(0,0.5,'migration distance (m), with phase lag')

In [90]:
# 7. Map and plot data by bends

plt.figure(figsize=(60,60))
           
plt.plot(x4,y4, 'y')
plt.plot(x1[3137:],y1[3137:],'b')
plt.axis('equal')
plt.scatter(x4[LZC], y4[LZC], c='r')
plt.scatter(x4[LZM], y4[LZM], c='k')  

for i, txt in enumerate(BEND):
    plt.annotate(txt, (x4[LZC][i], y4[LZC][i]), color='r', size='small')
for i, txt in enumerate(BEND):
    plt.annotate(txt, (x4[LZM][i], y4[LZM][i]), color='k', size='small')
plt.ylabel('distance from south to north (m)', fontsize =12); 
plt.xlabel('distance from east to west (m)',fontsize =12)
plt.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))

In [138]:
# 8. Let's plot data by bends

W = 174

fig, ax1 = plt.subplots(figsize=(16,4))

y1g = .45 # Top for curv
y2g = 0.0 # Zero for curv
y3g = -0.615625 # Zero for migr
y4g = -1.1 # Bottom for curv

for i in range(1,len(LZC)-1,2):
    xcoords = [LZC_s[i],LZC_s[i+1],LZC_s[i+1],LZM_s[i+1],LZM_s[i+1],LZM_s[i],LZM_s[i],LZC_s[i]]
    ycoords = [y1g,y1g,y2g,y3g,y4g,y4g,y3g,y2g]
    ax1.fill(xcoords,ycoords,color=[0.85,0.85,0.85],zorder=0) 
        
offset = 10
deltas = 25.0

ax1.fill_between(s, 0, curv*W) 
ax2 = ax1.twinx()
ax2.fill_between(s, 0, migr_dist, facecolor='green')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y4g,y1g)
ax2.set_ylim(-500,1100)
ax1.set_xlim(0,s[-1])
   
ax1.set_ylabel('W/r (m)')
ax2.set_ylabel('migration distance (m)')
ax1.set_xlabel("distance along channel axis (m)")

plt.tight_layout()
    

# Timestep B. Horizon 4 to Horizon 2 (to pick bends upstream)

In [41]:
# 1. Compute, Smooth Curvatures

curv,s = compute_curvature(x2,y2) # Horizon 1

from scipy.signal import savgol_filter#Plot Smoothed and Raw Curvature
curv_s = savgol_filter(curv,51,3) # mess with "51" or middle figure to play w smoothing (changed to 31)

curv,s = compute_curvature(x4,y4) # Horizon 4

from scipy.signal import savgol_filter#Plot Smoothed and Raw Curvature
curv_s = savgol_filter(curv,51,3) # mess with "51" or middle figure to play w smoothing (changed to 31)

In [42]:
# 2. Compute, Smooth Migration Distance

years = 1.0 #to get migration distance because we dont know the rate
migr_dist, migr_sign, p, q = get_migr_dist(x4,y4,x2,y2,years)

migr_dist = medfilt(savgol_filter(migr_dist,51,3),kernel_size=5) # smoothing
curv,s = compute_curvature(x4,y4)

curv = medfilt(savgol_filter(curv,51,3),kernel_size=5) # smoothing

loc_zero_curv_s, loc_zero_curv = zerocross1d(s, curv, getIndices=True)
loc_zero_migr_s, loc_zero_migr = zerocross1d(s, migr_dist, getIndices=True)

In [43]:
#  Run to crop out incomplete contour data

migr_dist = migr_dist[:3389]
curv = curv[:3389]
s = s[:3389] 

In [107]:
# 3. Import bends* from CSV 

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/Joshua_hz4_to_hz2_inflection_and_zero_migration_indices.csv', usecols=[0,1,2,3,4])
df.columns = ['bend','index_inflection_point','index_zero_migration','s_coordinate_index_ip','s_coordinate_index_zm']
                 
BEND = np.array(df['bend'])
LZC = np.array(df['index_inflection_point'])
LZM = np.array(df['index_zero_migration'])
LZC_s = np.array(df['s_coordinate_index_ip'])
LZM_s = np.array(df['s_coordinate_index_zm'])

# Editing bend assignments, first by matching bend no 1 to Hz 2 to Hz1

BEND=BEND+26  # add [amount] to bend 0 to account for upstream curvatures, match to 2 to 1

LZC=LZC.astype(int)
LZM=LZM.astype(int)

In [34]:
# 4. Find lag

# Rough estimate

print np.mean((LZC-LZM)*deltas)
print np.mean(LZC-LZM)


-641.9354838709677
-25.677419354838708


In [43]:
#  Iterate lag estimate for final lag


#Without lag
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, _ = pearsonr(curv, migr_dist)
print('Pearsons correlation: %.3f' % corr)

#With lag!
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, _ = pearsonr(curv[:-27], migr_dist[27:])
print('Pearsons correlation: %.3f' % corr)

# curv_tsb_lagged = curv[:-27]
# migr_tsb_lagged = migr_dist[27:]

Pearsons correlation: 0.544
Pearsons correlation: 0.755


In [47]:
# 5. Correlation plot

plt.figure(figsize=(8,8))
sns.kdeplot(curv[:-27], migr_dist[27:],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)

## Run to crop data
# plt.scatter(curv[3389:-27][::20],migr_dist[27+3389:][::20],c='y',s=10) 
# plt.scatter(curv[:3389-27][::20],migr_dist[27:3389][::20],c='k',s=10)

plt.scatter(curv[:-27][::20],migr_dist[27:][::20],c='k',s=10)
plt.xlabel('curvature (1/m)', fontsize=14)
plt.ylabel('migration distance (m), with phase lag', fontsize=14)


Text(0,0.5,'migration distance (m), with phase lag')

In [108]:
# 6. Map bends, cropped

plt.figure(figsize=(60,60))
           
plt.plot(x4,y4, 'y')
plt.plot(x2[3018:],y2[3018:],'g')
plt.axis('equal')
plt.scatter(x4[LZC], y4[LZC], c='r')
plt.scatter(x4[LZM], y4[LZM], c='k')  

for i, txt in enumerate(BEND):
    plt.annotate(txt, (x4[LZC][i], y4[LZC][i]), color='r', size='small')
for i, txt in enumerate(BEND):
    plt.annotate(txt, (x4[LZM][i], y4[LZM][i]), color='k', size='small')

plt.ylabel('distance from south to north (m)', fontsize =12); 
plt.xlabel('distance from east to west (m)',fontsize =12)
plt.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))


In [119]:
# 7. Plot data by bends

W = 174

fig, ax1 = plt.subplots(figsize=(16,4))

y1g = .45 # Top for curv
y2g = 0.0 # Zero for curv
y3g = -0.615625 # Zero for migr
y4g = -1.1 # Bottom for curv

for i in range(1,len(LZC)-1,2):
    xcoords = [LZC_s[i],LZC_s[i+1],LZC_s[i+1],LZM_s[i+1],LZM_s[i+1],LZM_s[i],LZM_s[i],LZC_s[i]]
    ycoords = [y1g,y1g,y2g,y3g,y4g,y4g,y3g,y2g]
    ax1.fill(xcoords,ycoords,color=[0.85,0.85,0.85],zorder=0) 
        
offset = 10
deltas = 25.0

ax1.fill_between(s, 0, curv*W) 
ax2 = ax1.twinx()
ax2.fill_between(s, 0, migr_dist, facecolor='green')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y4g,y1g)
ax2.set_ylim(-500,1100)
ax1.set_xlim(0,s[-1])

ax1.set_ylabel('W/r (m)')
ax2.set_ylabel('migration distance (m)')
ax1.set_xlabel("distance along channel axis (m)")

plt.tight_layout()

# Timestep C. Horizon 3 to Horizon 1

In [55]:
# 1. Compute, Smooth Curvature

from scipy.signal import savgol_filter#Plot Smoothed and Raw Curvature

curv,s = compute_curvature(x3,y3) # Horizon 3
curv_s = savgol_filter(curv,51,3) # mess with "51" or middle figure to play w smoothing (changed to 31)


In [56]:
# 2. Compute, Smooth Migration Distance

years = 1.0 #To get migration distance because we don't know the rate
migr_dist, migr_sign, p, q = get_migr_dist(x3,y3,x1,y1,years)

migr_dist = medfilt(savgol_filter(migr_dist,51,3),kernel_size=5) # smoothing
curv,s = compute_curvature(x3,y3)

curv = medfilt(savgol_filter(curv,51,3),kernel_size=5) # smoothing

loc_zero_curv_s, loc_zero_curv = zerocross1d(s, curv, getIndices=True)
loc_zero_migr_s, loc_zero_migr = zerocross1d(s, migr_dist, getIndices=True)

In [57]:
#  Run to crop out incomplete contour data

migr_dist = migr_dist[:5055]
curv = curv[:5055]
s = s[:5055] 

In [59]:
# 3. Import bends* from CSV 

# * To edit, see "bend_picker" in /Desktop/Thesis Data Processing and change file description below to "..._NEW"
# For smoothing factor on curv at '31' use v2, for '51' use  v3

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/Joshua_hz3_to_hz1_inflection_and_zero_migration_indices.csv', usecols=[0,1,2,3,4])
df.columns = ['bend','index_inflection_point','index_zero_migration','s_coordinate_index_ip','s_coordinate_index_zm']
                 
BEND = np.array(df['bend'])
LZC = np.array(df['index_inflection_point'])
LZM = np.array(df['index_zero_migration'])
LZC_s = np.array(df['s_coordinate_index_ip'])
LZM_s = np.array(df['s_coordinate_index_zm'])

BEND=BEND+13  # add [amount] to bend 0 to account for upstream curvatures, match to 2 to 1
LZC=LZC.astype(int)
LZM=LZM.astype(int)


In [60]:
# 4. Find lag

# Rough estimate

print np.median((LZC-LZM)*deltas)
print np.median(LZC-LZM)


-725.0
-29.0


In [62]:
# Iterate to find final lag

#Without lag
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, _ = pearsonr(curv, migr_dist)
print('Pearsons correlation: %.3f' % corr)

#With lag!
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, _ = pearsonr(curv[:-27], migr_dist[27:])
print('Pearsons correlation: %.3f' % corr)

print 27*deltas # Convert lag from index points to meters

Pearsons correlation: 0.452
Pearsons correlation: 0.752


In [67]:
# 5. Correlation plot

plt.figure(figsize=(8,8))
sns.kdeplot(curv[:-27], migr_dist[27:],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)

plt.scatter(curv[:-27][::20],migr_dist[27:][::20],c='k',s=10)

## Run to color incomplete data
# plt.scatter(curv[:5055-28][::20],migr_dist[28:5055][::20],c='k',s=10)
# plt.scatter(curv[5055:-28][::20],migr_dist[28+5055:][::20],c='y',s=10)

plt.xlabel('curvature (1/m)', fontsize=14)
plt.ylabel('migration distance (m), with phase lag', fontsize=14)


Text(0,0.5,'migration distance (m), with phase lag')

In [115]:
# 6. Map bends, cropped

plt.figure(figsize=(60,60))
           
plt.plot(x3,y3, 'c')
plt.plot(x1[1437:],y1[1437:],'b')
plt.axis('equal')
plt.scatter(x3[LZC], y3[LZC], c='r')
plt.scatter(x3[LZM], y3[LZM], c='k')  

for i, txt in enumerate(BEND):
    plt.annotate(txt, (x3[LZC][i], y3[LZC][i]), color='r', size='small')
for i, txt in enumerate(BEND):
    plt.annotate(txt, (x3[LZM][i], y3[LZM][i]), color='k', size='small')
plt.ylabel('distance from south to north (m)', fontsize =12); 
plt.xlabel('distance from east to west (m)',fontsize =12)
plt.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))


In [115]:
# 7. Let's plot data by bends

W = 174

fig, ax1 = plt.subplots(figsize=(16,4))

y1g = .45 # Top for curv
y2g = 0.0 # Zero for curv
y3g = -0.615625 # Zero for migr
y4g = -1.1 # Bottom for curv

for i in range(0,len(LZC)-1,2):
    xcoords = [LZC_s[i],LZC_s[i+1],LZC_s[i+1],LZM_s[i+1],LZM_s[i+1],LZM_s[i],LZM_s[i],LZC_s[i]]
    ycoords = [y1g,y1g,y2g,y3g,y4g,y4g,y3g,y2g]
    ax1.fill(xcoords,ycoords,color=[0.85,0.85,0.85],zorder=0) 
        
offset = 10
deltas = 25.0

ax1.fill_between(s, 0, curv*W)
ax2 = ax1.twinx()
ax2.fill_between(s, 0, migr_dist, facecolor='green')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y4g,y1g)
ax2.set_ylim(-450,990)
ax1.set_xlim(0,s[-1])
   
ax1.set_ylabel('W/r (m)')
ax2.set_ylabel('migration distance (m)')
ax1.set_xlabel("distance along channel axis (m)")

plt.tight_layout()
    